# **IMPORT NECESSARY PAKAGES**

In [ ]:
## Importing libraries
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, Input, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import to_categorical

from google.colab.patches import cv2_imshow

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# **BUILDING MODEL**

In [ ]:
#Using pre-trained model
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
#Define class Model
class Model(tf.keras.models.Sequential):
  def __init__(self):
    super().__init__()
    self.model = tf.keras.models.Sequential()


  def build(self):
    for layer in self.model.layers:
      self.model.layer.trainable = False
    self.model.add(baseModel)
    self.model.add(AveragePooling2D(pool_size=(7, 7)))
    self.model.add(Flatten())
    self.model.add(Dense(128, activation= 'relu'))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(2,activation='softmax'))

  def summary(self):
    return self.model.summary()

  def compile(self):
    self.model.compile(optimizer="adam",loss="binary_crossentropy",metrics =["accuracy"])

  def fit(self, datagen, X_train, y_train, X_test, y_test, epoch):
    self.model.fit(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train) // 32,validation_data=(X_test, y_test),validation_steps=len(X_test) // 32,epochs=epoch)

  def evaluate(self, X_test, y_test):
    evaluation = self.model.evaluate(X_test, y_test)
    print("Total loss: ", evaluation[0])
    print("Model accuracy: ", evaluation[1])

  def predict(self, X_test):
    return self.model.predict(X_test)

  def save(self, name= "model"):
    self.model.save(f"{name}.h5")

# **PROCESSING DATA**

In [ ]:
data = []
label = []

In [ ]:
#Path to dataset
path = "/content/drive/MyDrive/IE221/project/data"

In [ ]:
#Processing data, read and convert image to array
for dir in os.listdir(path):
  data_path = os.path.join(path, dir)
  for img in tqdm(os.listdir(data_path)):
    img_path = os.path.join(data_path, img)
    img = load_img(img_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)

    data.append(img)
    label.append(dir)

 27%|██▋       | 520/1915 [00:33<00:16, 83.45it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 1918/1918 [00:33<00:00, 56.47it/s] 


In [ ]:
#Transform label
lb = LabelBinarizer()
label = lb.fit_transform(label)
label = to_categorical(label)

In [ ]:
data = np.array(data, dtype="float32")
labels = np.array(label)

In [ ]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [ ]:
#Data augmentation
datagen = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

# **TRAINING DATA**

In [ ]:
model = Model()

In [ ]:
model.build()

In [ ]:
model.compile()

In [ ]:
his = model.fit(datagen, X_train, y_train, X_test, y_test, 30)

Epoch 1/30
95/95 [==============================] - 94s 461ms/step - loss: 0.1328 - accuracy: 0.9591 - val_loss: 0.6583 - val_accuracy: 0.9413
Epoch 2/30
95/95 [==============================] - 38s 403ms/step - loss: 0.0596 - accuracy: 0.9842 - val_loss: 0.4128 - val_accuracy: 0.9661
Epoch 3/30
95/95 [==============================] - 39s 410ms/step - loss: 0.0611 - accuracy: 0.9799 - val_loss: 0.8259 - val_accuracy: 0.9309
Epoch 4/30
95/95 [==============================] - 39s 408ms/step - loss: 0.0723 - accuracy: 0.9792 - val_loss: 3.2018 - val_accuracy: 0.8422
Epoch 5/30
95/95 [==============================] - 38s 395ms/step - loss: 0.0707 - accuracy: 0.9786 - val_loss: 2.2712 - val_accuracy: 0.7392
Epoch 6/30
95/95 [==============================] - 38s 395ms/step - loss: 0.0396 - accuracy: 0.9875 - val_loss: 0.7821 - val_accuracy: 0.9257
Epoch 7/30
95/95 [==============================] - 40s 417ms/step - loss: 0.0362 - accuracy: 0.9911 - val_loss: 8.0732 - val_accuracy: 0.5997

In [ ]:
model.evaluate(X_test, y_test)

24/24 [==============================] - 3s 108ms/step - loss: 0.9078 - accuracy: 0.9387
Total loss:  0.9077818989753723
Model accuracy:  0.9387223124504089


In [ ]:
pred = model.predict(X_test)

24/24 [==============================] - 2s 44ms/step


In [ ]:
print(classification_report(y_test.argmax(axis=1), pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93       383
           1       0.89      1.00      0.94       384

    accuracy                           0.94       767
   macro avg       0.95      0.94      0.94       767
weighted avg       0.95      0.94      0.94       767



In [ ]:
model.save("MobileNet")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 average_pooling2d (AverageP  (None, 1, 1, 1280)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 2